In [6]:
import os
import requests
import base64
from dotenv import load_dotenv

# Set up Azure OpenAI
load_dotenv("credentials.env")

api_key="19bac87c75964f75af65deabdbe43ffa"

IMAGE_PATH = "data/complex_report_screenshot.png"

encoded_image = base64.b64encode(open(IMAGE_PATH, 'rb').read()).decode('ascii')
headers = {
    "Content-Type": "application/json",
    "api-key": api_key,
}

# Payload for the request
payload = {
  "messages": [
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "You are an AI assistant that extract information from images."
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{encoded_image}"
          }
        },
        {
          "type": "text",
          "text":"explain this image and give me launch date, fund size and risk class as key value pairs."
        }
      ]
    }
  ],
  "temperature": 0.7,
  "top_p": 0.95,
  "max_tokens": 800
}

endpoint = "XXX"

# Send request
try:
    response = requests.post(endpoint, headers=headers, json=payload)
    response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
except requests.RequestException as e:
    raise SystemExit(f"Failed to make the request. Error: {e}")

# Handle the response as needed (e.g., print or process)
print(response.json())

{'choices': [{'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}, 'finish_reason': 'stop', 'index': 0, 'message': {'content': 'This image is a detailed fund fact sheet for the "Principal - MPF Conservative Fund (CAP)," provided by Principal MPF. It contains various sections that inform an investor about the fund\'s characteristics, performance, investment allocations, and holdings. The document includes:\n\n- **Fund Details**: Basic information such as fund descriptor, management company, launch date, and size.\n- **Investment Objective**: Describes the purpose and investment goals of the fund.\n- **Balance of Investments**: Explains how the fund achieves its investment objectives.\n- **Fund Performance**: Displays the performance metrics over different time periods.\n- **Fund Commentary**: Offers a market

In [9]:
import json
result = json.loads(response.text)["choices"][0]["message"]["content"]

In [10]:
result

'This image is a detailed fund fact sheet for the "Principal - MPF Conservative Fund (CAP)," provided by Principal MPF. It contains various sections that inform an investor about the fund\'s characteristics, performance, investment allocations, and holdings. The document includes:\n\n- **Fund Details**: Basic information such as fund descriptor, management company, launch date, and size.\n- **Investment Objective**: Describes the purpose and investment goals of the fund.\n- **Balance of Investments**: Explains how the fund achieves its investment objectives.\n- **Fund Performance**: Displays the performance metrics over different time periods.\n- **Fund Commentary**: Offers a market commentary and the fund manager\'s insights.\n- **Fund Allocation by Asset Class**: Shows a pie chart distribution of investments across different asset classes.\n- **Top 10 Holdings**: Lists the top assets held within the fund.\n\nHere are the key details requested:\n\n- **Launch Date**: 1/12/2000\n- **Fun

# JSON FORMAT OUTPUT

In [11]:
import datetime
import json
import openai
import os
import sys

from openai import AzureOpenAI
from dotenv import load_dotenv
from pprint import pprint

In [13]:
# Set up Azure OpenAI
load_dotenv("credentials.env")

openai.api_type = "azure"
    
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-02-01",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

In [16]:
completion = client.chat.completions.create(
    model="gpt-4-0125-Preview", #"gpt-4turbo-0409",
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant. Your response should be in JSON format.",
        },
        {
            "role": "user",
            "content": "List me 10 top movies from the 70's with the title, the director, the year \
        and the main actors",
        },
    ],
    response_format={"type": "json_object"},
)

In [17]:
print(completion)

ChatCompletion(id='chatcmpl-9yb4ipxZKfektthNlGlh2MZzm1BE6', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "movies": [\n    {\n      "title": "The Godfather",\n      "director": "Francis Ford Coppola",\n      "year": 1972,\n      "main_actors": ["Marlon Brando", "Al Pacino", "James Caan"]\n    },\n    {\n      "title": "Star Wars",\n      "director": "George Lucas",\n      "year": 1977,\n      "main_actors": ["Mark Hamill", "Harrison Ford", "Carrie Fisher"]\n    },\n    {\n      "title": "Jaws",\n      "director": "Steven Spielberg",\n      "year": 1975,\n      "main_actors": ["Roy Scheider", "Robert Shaw", "Richard Dreyfuss"]\n    },\n    {\n      "title": "A Clockwork Orange",\n      "director": "Stanley Kubrick",\n      "year": 1971,\n      "main_actors": ["Malcolm McDowell", "Patrick Magee", "Adrienne Corri"]\n    },\n    {\n      "title": "Taxi Driver",\n      "director": "Martin Scorsese",\n      "year": 1976,\n      "mai

In [18]:
completion.usage

CompletionUsage(completion_tokens=547, prompt_tokens=50, total_tokens=597)

In [19]:
json_answer = json.loads(completion.choices[0].message.content)
pprint(json_answer)

{'movies': [{'director': 'Francis Ford Coppola',
             'main_actors': ['Marlon Brando', 'Al Pacino', 'James Caan'],
             'title': 'The Godfather',
             'year': 1972},
            {'director': 'George Lucas',
             'main_actors': ['Mark Hamill', 'Harrison Ford', 'Carrie Fisher'],
             'title': 'Star Wars',
             'year': 1977},
            {'director': 'Steven Spielberg',
             'main_actors': ['Roy Scheider', 'Robert Shaw', 'Richard Dreyfuss'],
             'title': 'Jaws',
             'year': 1975},
            {'director': 'Stanley Kubrick',
             'main_actors': ['Malcolm McDowell',
                             'Patrick Magee',
                             'Adrienne Corri'],
             'title': 'A Clockwork Orange',
             'year': 1971},
            {'director': 'Martin Scorsese',
             'main_actors': ['Robert De Niro',
                             'Jodie Foster',
                             'Cybill Shepherd'],

In [20]:
json_answer["movies"][0]

{'title': 'The Godfather',
 'director': 'Francis Ford Coppola',
 'year': 1972,
 'main_actors': ['Marlon Brando', 'Al Pacino', 'James Caan']}

In [21]:
for i in range(len(json_answer["movies"])):
    print(i+1, json_answer["movies"][i]["title"], json_answer["movies"][i]["year"])

1 The Godfather 1972
2 Star Wars 1977
3 Jaws 1975
4 A Clockwork Orange 1971
5 Taxi Driver 1976
6 The Exorcist 1973
7 Apocalypse Now 1979
8 Chinatown 1974
9 Rocky 1976
10 One Flew Over the Cuckoo's Nest 1975
